In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Input, Dense, Reshape, Permute, RepeatVector, Multiply, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, RepeatVector, Permute, Multiply, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import time

2024-07-31 16:00:07.445699: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 16:00:07.445830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 16:00:07.612495: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Define the paths
train_dir = '/kaggle/input/mouth-and-oral-disease-mod-dataset/Mouth and Oral Disease (MOD) dataset/Training'
test_dir = '/kaggle/input/mouth-and-oral-disease-mod-dataset/Mouth and Oral Disease (MOD) dataset/Testing'
val_dir = '/kaggle/input/mouth-and-oral-disease-mod-dataset/Mouth and Oral Disease (MOD) dataset/Validation'

# Define parameters
img_size = (224, 224)  # EfficientNet expects images of size 224x224
batch_size = 64
epochs = 10
learning_rate = 0.001
input_shape = (224,224,3)

In [3]:
# generator
train_generator = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode = 'categorical',
    image_size = img_size,
    batch_size = batch_size,
    )

validation_generator = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode = 'categorical',
    image_size = img_size,
    batch_size = batch_size,
    shuffle = True
)

test_generator = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode = 'categorical',
    image_size = img_size,
    batch_size = batch_size)


# Determine the number of classes

class_names = np.array([item for item in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, item))])
# print(class_names)
num_classes = len(class_names)



Found 3087 files belonging to 7 classes.
Found 1028 files belonging to 7 classes.
Found 1028 files belonging to 7 classes.


In [4]:
class NovelSelfAttentionBlock(Layer):
    def __init__(self, units=128):
        super(NovelSelfAttentionBlock, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        self.dense = Dense(self.units, activation='relu')
        self.flatten = Flatten()
        self.softmax = tf.keras.layers.Softmax()
        self.repeat_vector = RepeatVector(input_shape[1])
        self.permute = Permute((2, 1))
        self.multiply = Multiply()
    
    def call(self, inputs):
        # Step 1: Compute attention weights
        Z = self.dense(inputs)
        flattened_Z = self.flatten(Z)
        attention_weights = self.softmax(flattened_Z)
        
        # Step 2: Modify attention weights
        repeated_attention_weights = self.repeat_vector(attention_weights)
        permuted_attention_weights = self.permute(repeated_attention_weights)
        
        # Step 3: Apply attention weights to the input tensor
        attended_output = self.multiply([inputs, permuted_attention_weights])
        
        # Return attended output
        return attended_output

def create_model(base_model0, base_model1, num_classes):
    input_tensor = Input(shape=input_shape)
    
    # Process with base models
    conv0 = base_model0(input_tensor)
    conv0 = GlobalAveragePooling2D()(conv0)
    
    conv1 = base_model1(input_tensor)
    conv1 = GlobalAveragePooling2D()(conv1)
    
    # Concatenate features
    concatenated_features = Concatenate()([conv0, conv1])
    
    # Apply self-attention block
    attention_output = NovelSelfAttentionBlock()(concatenated_features)
    
    # Flatten the output from the attention block
    flattened_output = Flatten()(attention_output)
    
    # Output layer
    output_layer = Dense(num_classes, activation='softmax')(flattened_output)

    model = Model(inputs=input_tensor, outputs=output_layer)
    return model

# Define paths and parameters
input_shape = (224, 224, 3)
base_model0 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)
base_model1 = EfficientNetB1(include_top=False, weights='imagenet', input_shape=input_shape)

for layer in base_model0.layers:
    layer.trainable = False

for layer in base_model1.layers:
    layer.trainable = False

num_classes = 7 
learning_rate = 0.001

def train_model(base_model0, base_model1, num_classes):
    model = create_model(base_model0, base_model1, num_classes)
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
#     model.summary()
    
    # Callbacks
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        ModelCheckpoint(f'best_model.keras', save_best_only=True)  # Use .keras extension
    ]

    # Train the model
    history = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        callbacks=callbacks
    )


    return model, history



16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
def calculate_metrics(model, generator):
    y_true = []
    y_pred = []
    
    for images, labels in generator:
        y_true.extend(np.argmax(labels.numpy(), axis=1))
        y_pred.extend(np.argmax(model.predict(images), axis=1))
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    return accuracy, precision, recall, f1

In [6]:
# Initialize lists to store metrics for 10 runs
train_metrics = []
test_metrics = []
train_times = []
test_times = []

# Run the model training and testing 10 times
for i in range(10):
    print(f"Run {i+1}/10")
    # Evaluate the model on training data
    
#     # Train the model
    start_time = time.time()
    model, history = train_model(base_model0, base_model1, num_classes)
    train_time = time.time() - start_time
    
#     start_time = time.time()
    train_accuracy, train_precision, train_recall, train_f1 = calculate_metrics(model, train_generator)
    train_metrics.append((train_accuracy, train_precision, train_recall, train_f1))
    print(train_metrics)
    train_time = time.time() - start_time
    train_times.append(train_time)


    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Training Precision: {train_precision:.4f}")
    print(f"Training Recall: {train_recall:.4f}")
    print(f"Training F1 Score: {train_f1:.4f}")
    print(f"Training Time: {train_time:.2f} seconds")
    
    
    start_time = time.time()
    test_accuracy, test_precision, test_recall, test_f1 = calculate_metrics(model, test_generator)
    test_metrics.append((test_accuracy, test_precision, test_recall, test_f1))
    test_time = time.time() - start_time
    test_times.append(test_time)
    
    print(f"Testing Accuracy: {test_accuracy:.4f}")
    print(f"Testing Precision: {test_precision:.4f}")
    print(f"Testing Recall: {test_recall:.4f}")
    print(f"Testing F1 Score: {test_f1:.4f}")
    print(f"Testing Time: {test_time:.2f} seconds")

# Calculate averages of metrics
avg_train_metrics = np.mean(train_metrics, axis=0)
avg_test_metrics = np.mean(test_metrics, axis=0)
avg_train_time = np.mean(train_times)
avg_test_time = np.mean(test_times)


print("--------------------------------------***************************************-------------------------------------")
print("Average Training Metrics:")
# print(f"TP: {avg_train_metrics[0]}, TN: {avg_train_metrics[0][1]}, FP: {avg_train_metrics[2]}, FN: {avg_train_metrics[3]}")
print(f"Accuracy: {avg_train_metrics[0]}, Precision: {avg_train_metrics[1]}, Recall: {avg_train_metrics[2]}, F1 Score: {avg_train_metrics[3]}")
print(f"Average Training Time: {avg_train_time} seconds")

print("\nAverage Testing Metrics:")
# print(f"TP: {avg_test_metrics[0]:.2f}, TN: {avg_test_metrics[1]:.2f}, FP: {avg_test_metrics[2]:.2f}, FN: {avg_test_metrics[3]:.2f}")
print(f"Average Testing Metrics - Accuracy: {avg_test_metrics[0]:.4f}, Precision: {avg_test_metrics[1]:.4f}, Recall: {avg_test_metrics[2]:.4f}, F1 Score: {avg_test_metrics[3]:.4f}")
print(f"Average Testing time: {avg_test_time:.2f} seconds")


Run 1/10
Epoch 1/10


I0000 00:00:1722441716.051931     118 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1722441716.172283     118 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5098 - loss: 1.3603   

W0000 00:00:1722441775.984385     117 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1722441789.121001     119 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1722441802.869835     117 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


49/49 ━━━━━━━━━━━━━━━━━━━━ 176s 2s/step - accuracy: 0.5126 - loss: 1.3534 - val_accuracy: 0.8424 - val_loss: 0.4838
Epoch 2/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 12s 251ms/step - accuracy: 0.8866 - loss: 0.3633 - val_accuracy: 0.9494 - val_loss: 0.1671
Epoch 3/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 13s 257ms/step - accuracy: 0.9708 - loss: 0.1019 - val_accuracy: 0.9737 - val_loss: 0.0877
Epoch 4/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 13s 264ms/step - accuracy: 0.9938 - loss: 0.0445 - val_accuracy: 0.9796 - val_loss: 0.0680
Epoch 5/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 13s 268ms/step - accuracy: 0.9928 - loss: 0.0313 - val_accuracy: 0.9864 - val_loss: 0.0528
Epoch 6/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 11s 226ms/step - accuracy: 0.9968 - loss: 0.0189 - val_accuracy: 0.9757 - val_loss: 0.0671
Epoch 7/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 11s 224ms/step - accuracy: 0.9950 - loss: 0.0199 - val_accuracy: 0.9844 - val_loss: 0.0618
Epoch 8/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 11s 222ms/step - accuracy: 0.9958 - loss: 0.0136 - val_accuracy: 0.9854 

...

W0000 00:00:1722444636.662696     119 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
2/2 ━━━━━━━━

W0000 00:00:1722444659.571042     116 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Testing Accuracy: 0.9874
Testing Precision: 0.9876
Testing Recall: 0.9874
Testing F1 Score: 0.9874
Testing Time: 9.51 seconds
--------------------------------------***************************************-------------------------------------
Average Training Metrics:
Accuracy: 0.9994169096209913, Precision: 0.9994191563941603, Recall: 0.9994169096209913,

W0000 00:00:1722444669.082222     118 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
